# Atividade Somativa 1

Para essa atividade foi escolhido o dataset **seoul_bike_data**. Ele contém dados horários sobre o aluguel de bicicletas públicas em Seul, capital da Coreia do Sul, cobrindo um período de um ano (de 1º de dezembro de 2017 a 30 de novembro de 2018). 

## Visão Geral do Dataset
O objetivo principal ao usar este dataset é prever a demanda por bicicletas alugadas (Rented Bike Count) com base nas outras variáveis disponíveis.

# Carregando o Dataset

Vamos carregar o dataset em um dataframe e analisar seus features.

O dataset é composto por 8.760 linhas (24 horas por dia × 365 dias) e 14 colunas. As colunas são:

| Nome da Coluna | Descrição | Tipo de Dado | Exemplo |
| :--- | :--- | :--- | :--- |
| **DateTime** | A data e hora do registro. | Objeto/Texto | 01/12/20172017-01-12 00:00:00 |
| **Day** | Dia do registro. | Numérico (Inteiro) | 01|
| **Weekday** | Dia da semana. | Numérico (Inteiro) | 7 |
| **Hour** | A hora do dia (0 a 23). | Numérico (Inteiro) | 0, 1, 2, ... |
| **Rented Bike Count** | **(Variável Alvo)** O número de bicicletas alugadas em uma determinada hora. | Numérico (Inteiro) | 254 |
| **Temperature(°C)** | A temperatura em graus Celsius. | Numérico (Float) | -5.2 |
| **Humidity(%)** | A umidade relativa do ar em porcentagem. | Numérico (Inteiro) | 37 |
| **Wind speed (m/s)** | A velocidade do vento em metros por segundo. | Numérico (Float) | 2.2 |
| **Visibility (10m)** | A visibilidade em uma escala de 10 metros (ex: 2000 significa 20km). | Numérico (Inteiro) | 2000 |
| **Dew point temp(°C)** | A temperatura do ponto de orvalho em graus Celsius. | Numérico (Float) | -17.6 |
| **Solar Radiation (MJ/m2)**| A radiação solar. Geralmente é zero durante a noite. | Numérico (Float) | 0.0 |
| **Rainfall(mm)** | A quantidade de chuva em milímetros. | Numérico (Float) | 0.0 |
| **Snowfall (cm)** | A quantidade de neve em centímetros. | Numérico (Float) | 0.0 |

## Bibliotecas 
Vamos começar importando as blibliotecas

In [1]:
import pandas as pd # importando o pandas para manipularmos o dataset
from ydata_profiling import ProfileReport # importando o pandas-profiling para fazer o profile do dataset
from sklearn.model_selection import train_test_split # função para dividir o dataset em treino e teste
import xgboost as xgb  # XGBoost para regressão
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.metrics import * # importando todas as funções de métricas do scikit-learn
from sklearn.metrics import mean_squared_error

# forçando a saída do pandas para que somente 3 dígitos sejam mostrados no lugar da notação científica
pd.options.display.float_format = '{:.3f}'.format

In [2]:
%matplotlib inline

Agora vamos importar o dataset para um dataframe

In [3]:
df_bike = pd.read_excel('seoul_bike_data.xlsx')
    
print("Amostra dos dados (primeiras 5 linhas):")
display(df_bike.head())

print("\nInformações gerais sobre o DataFrame:")
df_bike.info()

#profile = ProfileReport(df_bike)
#profile.to_file("relatorio.html")

Amostra dos dados (primeiras 5 linhas):


,DateTime,Day,Weekday,Hour,Rented Bike Count,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm)
0,2017-01-12 00:00:00,12,5,0,254,-5.200,37,2.200,2000,-17.600,0.000,0.000,0.000
1,2017-01-12 01:00:00,12,5,1,204,-5.500,38,0.800,2000,-17.600,0.000,0.000,0.000
2,2017-01-12 02:00:00,12,5,2,173,-6.000,39,1.000,2000,-17.700,0.000,0.000,0.000
3,2017-01-12 03:00:00,12,5,3,107,-6.200,40,0.900,2000,-17.600,0.000,0.000,0.000
4,2017-01-12 04:00:00,12,5,4,78,-6.000,36,2.300,2000,-18.600,0.000,0.000,0.000



Informações gerais sobre o DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DateTime                   8760 non-null   object 
 1   Day                        8760 non-null   int64  
 2   Weekday                    8760 non-null   int64  
 3   Hour                       8760 non-null   int64  
 4   Rented Bike Count          8760 non-null   int64  
 5   Temperature(°C)            8760 non-null   float64
 6   Humidity(%)                8760 non-null   int64  
 7   Wind speed (m/s)           8760 non-null   float64
 8   Visibility (10m)           8760 non-null   int64  
 9   Dew point temperature(°C)  8760 non-null   float64
 10  Solar Radiation (MJ/m2)    8760 non-null   float64
 11  Rainfall(mm)               8760 non-null   float64
 12  Snowfall (cm)              8760 non-null   float64
dtypes: float6

## Seleção de Atributos
Ao analisando a tabela de correlação estre as variáveis com a ferramenta **ydata_profiling**, foi possível notar o seguinte:

### Análise de Redundância (Multicolinearidade)

Este é o primeiro passo: encontrar atributos que medem quase a mesma coisa.

*   **`Temperature(°C)` vs. `Dew point temperature(°C)`: Correlação de 0.912**
    *   **Análise:** Este é um valor de correlação **extremamente alto**. Como esperado, a temperatura e a temperatura do ponto de orvalho estão fortemente ligadas. Manter ambas no modelo é redundante e pode confundir alguns algoritmos.
    *   **Ação Recomendada:** **remover uma dessas duas colunas**. Mas qual? Para decidir, olhamos a correlação de cada uma com a nossa variável alvo, `Rented Bike Count`.
        *   Correlação de `Temperature` com `Rented Bike Count` = **0.565**
        *   Correlação de `Dew point temperature` com `Rented Bike Count` = **0.374**
    *   **Decisão:** A `Temperature(°C)` tem uma correlação muito mais forte com o número de bicicletas alugadas. Portanto, **mantenha `Temperature(°C)` e removemos `Dew point temperature(°C)`**.

### Análise do Poder Preditivo (Relação com `Rented Bike Count`)

Agora, vamos focar na linha/coluna `Rented Bike Count` para ver quais atributos são os melhores preditores.

**Correlações Positivas (Quando estes aumentam, a demanda tende a aumentar):**

*   **`Temperature(°C)` (0.565):** Esta é a **correlação mais forte** com a variável alvo. Confirma a intuição de que a demanda por bicicletas aumenta significativamente com a temperatura. Este é o seu preditor mais importante.
*   **`Hour` (0.389):** Uma correlação positiva forte. Isso indica que, ao longo do dia (das 0h às 23h), há uma tendência geral de aumento na demanda, provavelmente devido aos picos da tarde/noite. A hora do dia é um preditor crucial.
*   **`Solar Radiation (MJ/m2)` (0.382):** Também uma correlação forte. Mais radiação solar significa um dia ensolarado e agradável, o que incentiva o uso de bicicletas.
*   **`Visibility (10m)` (0.176):** Correlação positiva, mas mais fraca. Melhor visibilidade (menos neblina) está associada a um leve aumento no número de aluguéis.
*   **`Wind speed (m/s)` (0.148):** Correlação positiva fraca. Isso é um pouco contraintuitivo, pois esperaríamos que ventos fortes desincentivassem o ciclismo. Essa correlação fraca pode indicar uma relação não-linear (ex: um pouco de vento é agradável, mas muito vento é ruim).

**Correlações Negativas (Quando estes aumentam, a demanda tende a diminuir):**

*   **`Rainfall(mm)` (-0.282):** Uma correlação negativa moderada. Como esperado, quando chove, as pessoas alugam menos bicicletas.
*   **`Snowfall (cm)` (-0.221):** Correlação negativa moderada. Similar à chuva, a neve também desincentiva o uso de bicicletas.
*   **`Humidity(%)` (-0.221):** Correlação negativa moderada. Alta umidade (clima abafado ou úmido) está associada a uma menor demanda.

**Correlações Próximas de Zero (Pouco Poder Preditivo Linear):**

*   **`Day` (0.067)** e **`Weekday` (0.062):** Essas colunas têm uma correlação linear muito fraca com a demanda. Isso **não significa que elas são inúteis!** Significa apenas que a relação não é uma linha reta simples (ex: a demanda não aumenta linearmente de segunda para domingo). O dia da semana (`Weekday`) é quase certamente um preditor muito importante devido à diferença de padrão entre dias úteis (picos de deslocamento) e fins de semana (uso para lazer). Modelos baseados em árvores (como RandomForest) conseguirão capturar essa relação complexa.

### Resumo e Próximos Passos

1. **Ação Imediata:** Remover a coluna `Dew point temperature(°C)` do dataset para eliminar a redundância.
2. **Preditores Mais Fortes:** Os atributos mais importantes são `Temperature(°C)`, `Hour` e `Solar Radiation (MJ/m2)`.
3. **Preditores Relevantes:** `Rainfall(mm)`, `Snowfall (cm)` e `Humidity(%)` também são importantes e devem ser mantidos.
4. **Não se Engane com Correlação Baixa:** Mantemos `Weekday` e `Hour`. Apesar da correlação linear baixa, eles contêm padrões cíclicos (diários e semanais) que são fundamentais para o problema e serão capturados por modelos mais sofisticados.
5. **Valores não numéricos:**: O algoritmo aprende com números, não podemos passar os atributos da coluna **DateTime**, então devemos excluir essa coluna da nossa base de dados.
   
## Normalização
Algoritmos que não são sensíveis à escala das features não exigem normalização.
Por que não é necessário?
Esses modelos tomam decisões com base em regras ou divisões nos dados, e a escala de uma feature não afeta onde essas divisões são feitas.

### Symbolists (Simbolistas):
Árvores de Decisão, Random Forest, XGBoost, LightGBM: Esta é a família de algoritmos mais forte para nosso problema. Eles funcionam fazendo "perguntas" sobre os dados, como "A Temperature é maior que 15°C?". Essa pergunta funciona da mesma forma, não importa se as outras features estão em escalas diferentes. Eles são imunes à escala das features.


# Divisão do dataset
Agora vamos dividir o dataset em três base de dados;
* Treinamento: Para treinar as árvores
* Validação (ou eval_set) para o Early Stopping: Para monitorar a performance e decidir quando parar. 
* Teste: Para a avaliação final.

## Early Stopping
O Early Stopping (Parada Antecipada) é uma técnica de regularização extremamente poderosa e prática, usada para encontrar o número ideal de **n_estimators** e, ao mesmo tempo, prevenir o overfitting. Permite que você defina um n_estimators bem alto e, então, monitora a performance do modelo em um conjunto de dados de validação, parando o treinamento automaticamente quando a performance para de melhorar.  

In [4]:
# Primeiro, dividimos em treino+validação (80%) e teste (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    df_bike.drop(['DateTime','Rented Bike Count', 'DateTime', 'Dew point temperature(°C)'], axis=1),   # remove as colunas desnecessárias 
    df_bike['Rented Bike Count'],   # Coluna que será usada como label de classificação
    test_size=0.2,  # informamos a porcentagem de divisão para a base de testes.
    random_state=35  # aqui informamos um "seed".
)

# Agora, dividimos o conjunto temporário em treino (75% de 80% = 60% do total) 
# e validação (25% de 80% = 20% do total)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=35
)

print("Dimensões dos conjuntos:")
print("dataset original", df_bike.shape)
print("X_train:", X_train.shape, "| y_train:", y_train.shape)
print("X_val:", X_val.shape, "  | y_val:", y_val.shape)
print("X_test:", X_test.shape, " | y_test:", y_test.shape)
print("-" * 30)

Dimensões dos conjuntos:
dataset original (8760, 13)
X_train: (5256, 10) | y_train: (5256,)
X_val: (1752, 10)   | y_val: (1752,)
X_test: (1752, 10)  | y_test: (1752,)
------------------------------


# Regressão
A regressão é usada para prever um valor numérico contínuo, neste caso, o número de bicicletas alugadas.
Como se aplica aqui? Você usa os outros atributos (hora, temperatura, umidade, estação do ano, etc.) como features (variáveis de entrada) para treinar um modelo de regressão. O objetivo do modelo é aprender a relação matemática entre essas features e o Rented Bike Count.

## XGBoost
Vamos usar o algoritmo **XGBoost**


In [5]:
# --- Treinamento com Early Stopping ---

# Instanciar o modelo com um n_estimators alto
xgbr = xgb.XGBRegressor(
    n_estimators=2000, # Um número alto, pois vamos parar antes se necessário. É o número total de especialistas (árvores de decisão)
    learning_rate=0.01, # É o peso dado (de 0 a 1) na confiança que o modelo tem da correção proposta pela nova árvore (n_estimators)
    max_depth=7,   # Uma profundidade de 5 significa que cada árvore pode fazer, no máximo, 3 níveis de perguntas "se-então-senão" para chegar a uma previsão.
    objective='reg:squarederror',   # define matematicamente o que significa "erro" para o seu problema específico. Define a função de perda (loss function) que o algoritmo tentará minimizar durante o treinamento.
    eval_metric='rmse',
    n_jobs=-1,  # Usa todos os núcleos de CPU disponíveis para acelerar o treino
    random_state=35,
    early_stopping_rounds=20 # Paciência de 10 rodadas
)

# Definir o conjunto de validação
eval_set = [(X_train, y_train), (X_val, y_val)]

print("Iniciando o treinamento com Early Stopping...")
# Treinar o modelo
xgbr.fit(
    X_train, y_train,
    eval_set=eval_set,
    verbose=False # Mostra o progresso do erro a cada rodada
)
print("Treinamento concluído!")
print(f"Melhor iteração encontrada: {xgbr.best_iteration}")

Iniciando o treinamento com Early Stopping...
Treinamento concluído!
Melhor iteração encontrada: 1904


Agora vamos analisar a predição usando a base de dados para teste

In [ ]:
print(f"score: {xgbr.score(X_test, y_test)}")
print(f"predição: {xgbr.predict(X_test)}")

In [ ]:
df_test = pd.DataFrame(X_test) 
df_test['Rented Bike Count_Real'] = y_test.values
df_test['Rented Bike Count_xgbr'] = xgbr.predict(X_test)
df_test

In [ ]:
previsoes = xgbr.predict(X_test)

# --- Comparar Previsões com Valores Reais ---
# Criar um novo DataFrame para comparar os resultados
resultados = pd.DataFrame({
    'Valor Real': y_test,
    'Valor Previsto': previsoes
})

# Adicionar uma coluna com a diferença (erro) para cada previsão
resultados['Erro'] = resultados['Valor Real'] - resultados['Valor Previsto']
resultados['Erro %'] = ((resultados['Valor Real'] - resultados['Valor Previsto']) * 100) / resultados['Valor Real']

print("\nAmostra da comparação entre valores reais e previstos:")
display(resultados.head(10))

In [6]:
previsoes_xgbr = xgbr.predict(X_test)

# --- Comparar Previsões com Valores Reais ---
# Criar um novo DataFrame para comparar os resultados
resultados = pd.DataFrame({
    'Valor Real': y_test,
    'Valor Previsto_xgbr': previsoes_xgbr
})

# Adicionar uma coluna com a diferença (erro) para cada previsão
resultados['Erro'] = resultados['Valor Real'] - resultados['Valor Previsto_xgbr']
resultados['Erro %'] = ((resultados['Valor Real'] - resultados['Valor Previsto_xgbr']) * 100) / resultados['Valor Real']

# calculando o rmse entre o valor real e o predito
rmse = mean_absolute_error(y_test, previsoes_xgbr)
# calculando o mape entre o valor real e o predito
mape = mean_absolute_percentage_error(y_test, previsoes_xgbr)


# Mostra metricas
print(f"score: {xgbr.score(X_test, y_test)}")
print(f"rmse: {rmse:.2f} bicicletas." )
print(f"mape: {mape:.2f}% de erro medio.")

print("\nAmostra da comparação entre valores reais e previstos:")
display(resultados.head(10))


score: 0.8476635813713074
rmse: 149.94 bicicletas.
mape: 72313196131647488.00% de erro medio.

Amostra da comparação entre valores reais e previstos:


,Valor Real,Valor Previsto_xgbr,Erro,Erro %
2665,513,519.120,-6.120,-1.193
7650,2200,1973.494,226.506,10.296
5558,638,719.284,-81.284,-12.740
6536,2129,1412.513,716.487,33.654
1187,547,543.983,3.017,0.552
5531,586,661.126,-75.126,-12.820
4244,2519,2264.136,254.864,10.118
4444,123,329.676,-206.676,-168.029
1843,1558,1657.319,-99.319,-6.375
7504,1344,1351.948,-7.948,-0.591
